In [6]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np


def plot(X, y):
    plt.close("all")
    if X.shape[1] == 1:
        plot_1d(X, y)
    elif X.shape[1] == 2:
        plot_2d(X, y)
    elif X.shape[1] == 3:
        plot_3d(X, y)


def plot_1d(X, y):
    plt.scatter(X, y, s=0.5)
    plt.show()


def plot_2d(X, y):
    y = y.flatten()
    fig = go.Figure()
    fig.add_trace(
        go.Scatter3d(
            x=X[:, 0],
            y=X[:, 1],
            z=y,
            mode="markers",
            marker=dict(size=1, color="blue"),
            name="Training Data",
        )
    )
    fig.update_layout(
        scene=dict(
            xaxis_title="X1",
            yaxis_title="X2",
            zaxis_title="y",
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        showlegend=True,
    )
    fig.show()


def plot_3d(X, y):
    y = y.flatten()  # Ensure y is a 1D array
    fig = go.Figure()
    fig.add_trace(
        go.Scatter3d(
            x=X[:, 0],  # X1 coordinate
            y=X[:, 1],  # X2 coordinate
            z=X[:, 2],  # X3 coordinate
            mode="markers",
            marker=dict(
                size=4,
                color=y,  # Color by y values
                colorscale="Viridis",  # Choose a color scale
                colorbar=dict(title="y values"),  # Add colorbar for reference
            ),
            name="Training Data",
        )
    )
    fig.update_layout(
        scene=dict(
            xaxis_title="X1",
            yaxis_title="X2",
            zaxis_title="X3",
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        showlegend=True,
    )
    fig.show()


def plot_oracle(X, y, mean, lower_bound, upper_bound):
    plt.close("all")
    if X.shape[1] == 1:
        plot_oracle_1d(X, y, mean, lower_bound, upper_bound)
    elif X.shape[1] == 2:
        plot_oracle_2d(X, y, mean, lower_bound, upper_bound)


def plot_oracle_1d(X, y, mean, lower_bound, upper_bound):
    line_width = 2
    marker_size = 1

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    X_sq = X.squeeze()
    mean_est_sq = mean.squeeze()
    sorted_indices = np.argsort(X_sq)
    X_sorted = X_sq[sorted_indices].squeeze()
    mean_est_sorted = mean_est_sq[sorted_indices].squeeze()

    upper_bound_sq = upper_bound.squeeze()
    lower_bound_sq = lower_bound.squeeze()
    upper_bound_sorted = upper_bound_sq[sorted_indices].squeeze()
    lower_bound_sorted = lower_bound_sq[sorted_indices].squeeze()

    ax.plot(X_sorted, mean_est_sorted, color="blue", linewidth=line_width)
    ax.fill_between(
        X_sorted,
        lower_bound_sorted,
        upper_bound_sorted,
        alpha=0.2,
        color="orange",
    )

    ax.scatter(X, y, s=marker_size, color="blue")

    ax.plot(
        X_sorted,
        upper_bound_sorted,
        color="orange",
        linewidth=line_width,
        ls="-",
    )
    ax.plot(
        X_sorted,
        lower_bound_sorted,
        color="orange",
        linewidth=line_width,
        ls="-",
    )

    ax.set_xlabel("x")
    ax.set_ylabel("y")

    plt.tight_layout()
    plt.show()


def plot_oracle_2d(X, y, mean, lower_bound, upper_bound):
    y = y.flatten()

    # Create 3D scatter plot
    fig = go.Figure()

    fig.add_trace(
        go.Scatter3d(
            x=X[:, 0],
            y=X[:, 1],
            z=y,
            mode="markers",
            marker=dict(size=1, color="blue"),
            name="Training Data",
        )
    )
    # Add mean estimation surface
    fig.add_trace(
        go.Mesh3d(
            x=X[:, 0],
            y=X[:, 1],
            z=mean.flatten(),
            color="blue",
            opacity=0.7,
            name="Mean Estimation",
        )
    )
    # Add upper bound surface
    fig.add_trace(
        go.Mesh3d(
            x=X[:, 0],
            y=X[:, 1],
            z=upper_bound.flatten(),
            color="orange",
            opacity=0.5,
            name="Upper Bound",
        )
    )
    # Add lower bound surface
    fig.add_trace(
        go.Mesh3d(
            x=X[:, 0],
            y=X[:, 1],
            z=lower_bound.flatten(),
            color="orange",
            opacity=0.5,
            name="Lower Bound",
        )
    )
    # Update layout
    fig.update_layout(
        scene=dict(
            xaxis_title="X1",
            yaxis_title="X2",
            zaxis_title="y",
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        showlegend=True,
    )
    fig.show()

In [38]:
from universalbands.data_generation import Synthetic

case_number = 5
sample_size = 100
sample_dim = 2
seed = 123


my_data = Synthetic(
    name=f"case_{case_number}",
    input_shape=(sample_size, sample_dim),
    output_shape=(sample_size, 1),
)

X, y = my_data.sample(seed=seed)

mean_oracle, lo_oracle, up_oracle = my_data.oracle(X=X, alpha=0.05)

#plot(X, y)
plot_oracle(X, y, mean_oracle, lo_oracle, up_oracle)

In [68]:
from GPy.kern import Matern52
from GPy.models.gp_regression import GPRegression

# Train the Gausian Process Regression.
kernel = Matern52(input_dim=sample_dim, variance=1.0, lengthscale=None, ARD=True, active_dims=None, name='Mat52')
gp_model = GPRegression(
    X,
    y,
    kernel,
    Y_metadata=None,
    normalizer=None,
    noise_var=1.0,
    mean_function=None,
)
gp_model.kern.variance.fix()  # Fixing the variance parameter.
gp_model.optimize_restarts(
    num_restarts=10, messages=False, verbose=True, max_iters=1000
)

# Retrieve posteriors parameters of the mean kernel.
posterior_lengthscale = list(gp_model.kern.lengthscale)
posterior_variance = gp_model.kern.variance[0]
posterior_nugget = gp_model.Gaussian_noise.variance[0]

# Reconstruct the norm of the function on the training parameters.
gamma = gp_model.posterior.woodbury_vector
K = gp_model.kern.K(X)
post_training_norm = np.matmul(gamma.T, np.matmul(K, gamma))[0][0]
print(post_training_norm)

theta_m = {
    "posterior_lengthscale": posterior_lengthscale,
    "posterior_variance": posterior_variance,
    "posterior_nugget": posterior_nugget,
    "posterior_training_norm": post_training_norm,
}

Optimization restart 1/10, f = 169.95752987265308
Optimization restart 2/10, f = 169.9575299075499
Optimization restart 3/10, f = 169.95753075817885
Optimization restart 4/10, f = 169.95752987350207
Optimization restart 5/10, f = 169.95752987267002
Optimization restart 6/10, f = 169.95752987314918
Optimization restart 7/10, f = 169.95752987264973
Optimization restart 8/10, f = 169.9575298731569
Optimization restart 9/10, f = 169.9575298726511
Optimization restart 10/10, f = 169.95752987293545
11.78211531176067


In [62]:
my_data = Synthetic(
    name=f"case_{case_number}",
    input_shape=(500, sample_dim),
    output_shape=(500, 1),
)

X_test, y_test = my_data.sample(seed=987)
mean_pred = gp_model.predict(X_test)


  index  |  GP_regression.Mat52.lengthscale  |  constraints  |  priors
  [0]    |                       0.87013600  |      +ve      |        
  [1]    |                       0.46962433  |      +ve      |        


In [63]:
gamma = gp_model.posterior.woodbury_vector
K = gp_model.kern.K(X)
post_training_norm = np.matmul(gamma.T, np.matmul(K, gamma))[0][0]
print(post_training_norm)

11.78211764279707


In [66]:
list(gp_model.Gaussian_noise.variance)

[1.4130771259763337]

In [59]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, mean_pred[0])

1.6818924543543021

In [64]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, mean_pred[0])

1.7151182224522517

In [12]:
from sklearn.gaussian_process import kernels
import numpy as np
kern = kernels.Matern(np.array([1, 0.5]))
from universalbands.data_generation import Synthetic

case_number = 5
sample_size = 100
sample_dim = 2
seed = 123


my_data = Synthetic(
    name=f"case_{case_number}",
    input_shape=(sample_size, sample_dim),
    output_shape=(sample_size, 1),
)

X, y = my_data.sample(seed=seed)

kern(X)

array([[1.        , 0.45154   , 0.02570851, ..., 0.61765322, 0.36844689,
        0.30669045],
       [0.45154   , 1.        , 0.06895517, ..., 0.77945684, 0.77207293,
        0.85941814],
       [0.02570851, 0.06895517, 1.        , ..., 0.07333634, 0.03757302,
        0.1105973 ],
       ...,
       [0.61765322, 0.77945684, 0.07333634, ..., 1.        , 0.50239448,
        0.65772751],
       [0.36844689, 0.77207293, 0.03757302, ..., 0.50239448, 1.        ,
        0.61405443],
       [0.30669045, 0.85941814, 0.1105973 , ..., 0.65772751, 0.61405443,
        1.        ]])